<img src="https://github.com/IKNL/guidelines/blob/master/resources/logos/iknl_nl.png?raw=true" width=200 align="right">

# `validation`
In this notebook, we validate our proposed federated GLM by comparing its performance against that of `R`'s `glm()` (i.e., centralized).

The families that are shown in this notebook are: 

* `gaussian(link = "identity")`: Linear regression
* `poisson(link = "log")`: Poisson regression
* `binomial(link = "logit")`: Logistic regression
* `rs.poi`: Custom GLM relative survival model with Poisson error


## Requirements
In order to be able to run it properly, make sure that the following libraries are installed:

* `vtg` - Basic `vantage6` tools. These will allow you to use the mock client. They can be installed with the command:

   `devtools::install_github('IKNL/vtg')`
   
   
   
* `vtg.glm` - The actual federated GLM algorithm. It can be installed with the command:

   `devtools::install_github(repo='IKNL/vantage6-algorithms', ref='glm', subdir='models/glm/src')`

## Preliminaries

In [1]:
# Imports
library('stargazer')


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




## Data
The `.csv` files for the data can be generated running the accompanying (Python) script `create_data.py`. Each family (i.e., model type) has its corresponding data. In all cases, they represent a horizontally-partitioned scenario (i.e., all parties have the same features from different individuals).

## Validation

#### The `validate()` function
This function compares the centralized and federated results. The most important values (coefficients, standard errors, $p$- and $z$-values are printed to the console using `stargazer`. Additionally, it check if other values (such as conversion, formula, family, etc.) are equal between them (shown as `TRUE`) as a sanity check.

In [2]:
validate <- function(results_centralized, results_federated_plain){
  
  results_centralized_summary <- summary.glm(results_centralized)
  results_federated <- vtg.glm::as.GLM(results_federated_plain)
  
  precision <- 10
  options(digits=precision)
  my_round <- function(x) {round(x, digits = 10)}
  
  validation_result <- list()
    
  # Coefficients
  coeffs_centralized = sapply(results_centralized$coefficients, my_round, USE.NAMES = TRUE)    
  coeffs_federated = sapply(results_federated_plain$coefficients, my_round, USE.NAMES = TRUE)
  
  df = data.frame(coeff_c = c(coeffs_centralized),
                  coeff_f = c(coeffs_federated))  
    
  # Standard error
  stderr_centralized = sapply(results_centralized_summary$coefficients[, 'Std. Error'], my_round, USE.NAMES = TRUE)
  stderr_federated = sapply(results_federated_plain$Std.Error, my_round, USE.NAMES = TRUE)
    
  df['stderr_c'] <- c(stderr_centralized)
  df['stderr_f'] <- c(stderr_federated)
  

  if(results_centralized_summary$family$family %in% c('poisson', 'binomial')) {letter='z'} 
  else if(results_centralized_summary$family$family=='gaussian') {letter='t'}
  
  # p-value
  pval_centralized = sapply(results_centralized_summary$coefficients[, sprintf("Pr(>|%s|)", letter)], my_round, USE.NAMES = TRUE)
  pval_federated = sapply(results_federated_plain$pvalue, my_round, USE.NAMES = TRUE)
   
  df['pvalues_c'] <- c(pval_centralized)
  df['pvalues_f'] <- c(pval_federated)
    
    
  # z-value
  zval_centralized = sapply(results_centralized_summary$coefficients[, sprintf("%s value", letter)], my_round, USE.NAMES = TRUE)
  zval_federated = sapply(results_federated_plain$zvalue, my_round, USE.NAMES = TRUE)
  
  df['zvalues_c'] <- c(zval_centralized)
  df['zvalues_f'] <- c(zval_federated)
    
  print("LaTeX")
  stargazer(df, type="latex", summary=FALSE)
  print("Text")
  stargazer(df, type="text", summary=FALSE)
    
    
  # Converged
  validation_result['converged'] = results_federated_plain$converged==results_centralized$converged
  print(validation_result['converged'])
    
  # Dispersion
  FL_dispersion = sapply(results_federated_plain$dispersion, my_round, USE.NAMES = TRUE)
  central_dispersion = sapply(results_centralized_summary$dispersion, my_round, USE.NAMES = TRUE)
  validation_result['dispersion'] = FL_dispersion==central_dispersion
  
  # Formula
  validation_result['formula'] = results_federated_plain$formula==results_centralized$formula
  
  # Family
  FL_family = jsonlite::toJSON(results_federated_plain$family, auto_unbox = TRUE, force = T)
  central_family = jsonlite::toJSON(results_centralized$family, auto_unbox = TRUE, force = T)
  validation_result['family'] = FL_family==central_family
  
  # null.deviance
  FL_nulldeviance = sapply(results_federated_plain$null.deviance, my_round, USE.NAMES = TRUE)
  central_nulldeviance = sapply(results_centralized$null.deviance, my_round, USE.NAMES = TRUE)
  validation_result['null.deviance'] = FL_nulldeviance==central_nulldeviance
  
  # DoF
  FL_dof = sapply(results_federated$df.residual, my_round, USE.NAMES = TRUE)
  central_dof = sapply(results_centralized$df.residual, my_round, USE.NAMES = TRUE)
  validation_result['df.residual'] = FL_dof==central_dof
  
  # null.dof
  FL_null = sapply(results_federated$df.null, my_round, USE.NAMES = TRUE)
  central_null = sapply(results_centralized$df.null, my_round, USE.NAMES = TRUE)
  validation_result['df.null'] = FL_null==central_null
    
  validation_result
}

### Linear
#### Centralized analysis

In [3]:
# Housekeeping
if(exists('results_centralized')){
    rm(results_centralized)
}
if(exists('results_federated_plain')){
    rm(results_federated_plain)
}

datasets <- list(
  read.csv('../data/linear_party1.csv'),
  read.csv('../data/linear_party2.csv'),
  read.csv('../data/linear_party3.csv')
)
datasets_combined <- do.call(rbind.data.frame, datasets)

results_centralized <- glm(data=datasets_combined, formula = y ~ x1 + x2, family=gaussian(link = 'identity'))

#### Federated analysis

In [4]:
client <- vtg::MockClient$new(datasets, 'vtg.glm')
results_federated_plain <- vtg.glm::dglm(client, formula = y ~ x1 + x2, family=gaussian(link = 'identity'), tol=1e-08, maxit=25)

-----------------------------------------
  Welcome to the vantage6 Infrastructure
-----------------------------------------




INFO  [11:56:30.386]  
INFO  [11:56:30.416] ############################################### 
INFO  [11:56:30.425] # Starting iteration 0 
INFO  [11:56:30.427] ############################################### 
INFO  [11:56:30.429]  
INFO  [11:56:30.431] 0.1 - RPC Node Beta 
DEBUG [11:56:30.434] ** Mocking call to "node_beta" ** 
DEBUG [11:56:30.437] Regular call 
DEBUG [11:56:30.439] there are 3 datasets ..  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:30.468] Log for site 1: 
DEBUG [11:56:30.471] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [11:56:30.447] Initializing node beta... 
DEBUG [11:56:30.454] First iteration. Initializing variables. 
DEBUG [11:56:30.457] Calculating the Betas.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


...

DEBUG [11:56:30.755] Log for site 1: 
DEBUG [11:56:30.757] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:30.747] Starting node deviance. 
DEBUG [11:56:30.752] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:30.773] Log for site 2: 
DEBUG [11:56:30.775] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:30.760] Starting node deviance. 
DEBUG [11:56:30.770] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:30.786] Log for site 3: 
DEBUG [11:56:30.789] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:30.778] Starting node deviance. 
DEBUG [11:56:30.784] Updating the variables for node deviance.  
DEBUG [11:56:30.791]   - [DONE] 
INFO  [11:56:30.794] 1.4 - Master deviance 
DEBUG [11:56:30.796] Starting master deviance. 
DEBUG [11:56:30.799]   - [DONE] 
INFO  [11:56:30.801] 1.5 - Termination conditions 
DEBUG [11:56:30.804]   - [CONVERGED] 
DEBUG [11:56:30.806] Preparing output 


#### Comparison

In [5]:
val_linear_res = validate(results_centralized, results_federated_plain)
print(val_linear_res)

[1] "LaTeX"

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Thu, May 05, 2022 - 11:56:30
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}} ccccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & coeff\_c & coeff\_f & stderr\_c & stderr\_f & pvalues\_c & pvalues\_f & zvalues\_c & zvalues\_f \\ 
\hline \\[-1.8ex] 
(Intercept) & $0.069$ & $0.069$ & $0.045$ & $0.045$ & $0.129$ & $0.129$ & $1.520$ & $1.520$ \\ 
x1 & $0.221$ & $0.221$ & $0.017$ & $0.017$ & $0$ & $0$ & $12.763$ & $12.763$ \\ 
x2 & $0.499$ & $0.499$ & $0.018$ & $0.018$ & $0$ & $0$ & $27.212$ & $27.212$ \\ 
\hline \\[-1.8ex] 
\end{tabular} 
\end{table} 
[1] "Text"

            coeff_c coeff_f stderr_c stderr_f pvalues_c pvalues_f zvalues_c zvalues_f
-------------------------------------------------------------------------------------
(Intercept)  0.069   0.069   0.045    0.045     0.129     0.129     1.52

### Poisson
#### Centralized

In [9]:
# Housekeeping
if(exists('results_centralized')){
    rm(results_centralized)
}
if(exists('results_federated_plain')){
    rm(results_federated_plain)
}

datasets <- list(
  read.csv('../data/poisson_party1.csv'),
  read.csv('../data/poisson_party2.csv'),
  read.csv('../data/poisson_party3.csv')
)
datasets_combined <- do.call(rbind.data.frame, datasets)

results_centralized <- glm(data=datasets_combined, formula = y ~ x1 + x2, family=poisson(link = "log"))

#### Federated

In [10]:
client <- vtg::MockClient$new(datasets, "vtg.glm")
results_federated_plain <- vtg.glm::dglm(client, formula = y ~ x1 + x2, family=poisson(link = "log"), tol=1e-08, maxit=25)

DEBUG [11:56:32.462] Initializing... 
INFO  [11:56:32.465]  
INFO  [11:56:32.467] ############################################### 
INFO  [11:56:32.470] # Starting iteration 0 
INFO  [11:56:32.472] ############################################### 
INFO  [11:56:32.475]  
INFO  [11:56:32.478] 0.1 - RPC Node Beta 
DEBUG [11:56:32.480] ** Mocking call to "node_beta" ** 
DEBUG [11:56:32.483] Regular call 
DEBUG [11:56:32.487] there are 3 datasets ..  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:32.502] Log for site 1: 
DEBUG [11:56:32.505] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [11:56:32.490] Initializing node beta... 
DEBUG [11:56:32.497] First iteration. Initializing variables. 
DEBUG [11:56:32.500] Calculating the Betas.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:32.522] Log for site 2: 
DEBUG [11:56:32.525] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [11:56:32.507] Initializing node beta... 
DEBUG [11:56:32.513] First iteration. Initializing variables. 
DEBUG [11:56:32.515] Calculating the Betas.  


...

DEBUG [11:56:33.281] Log for site 1: 
DEBUG [11:56:33.284] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:33.267] Starting node deviance. 
DEBUG [11:56:33.279] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:33.295] Log for site 2: 
DEBUG [11:56:33.298] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:33.286] Starting node deviance. 
DEBUG [11:56:33.292] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:33.311] Log for site 3: 
DEBUG [11:56:33.314] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:33.301] Starting node deviance. 
DEBUG [11:56:33.308] Updating the variables for node deviance.  
DEBUG [11:56:33.316]   - [DONE] 
INFO  [11:56:33.319] 4.4 - Master deviance 
DEBUG [11:56:33.321] Starting master deviance. 
DEBUG [11:56:33.323]   - [DONE] 
INFO  [11:56:33.327] 4.5 - Termination conditions 
DEBUG [11:56:33.330]   - [CONVERGED] 
DEBUG [11:56:33.333] Preparing output 


#### Comparison

In [11]:
val_poisson_res = validate(results_centralized, results_federated_plain)
print(val_poisson_res)

[1] "LaTeX"

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Thu, May 05, 2022 - 11:56:33
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}} ccccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & coeff\_c & coeff\_f & stderr\_c & stderr\_f & pvalues\_c & pvalues\_f & zvalues\_c & zvalues\_f \\ 
\hline \\[-1.8ex] 
(Intercept) & $0.595$ & $0.595$ & $0.021$ & $0.021$ & $0$ & $0$ & $28.465$ & $28.465$ \\ 
x1 & $0.269$ & $0.269$ & $0.007$ & $0.007$ & $0$ & $0$ & $37.220$ & $37.220$ \\ 
x2 & $0.446$ & $0.446$ & $0.007$ & $0.007$ & $0$ & $0$ & $62.566$ & $62.566$ \\ 
\hline \\[-1.8ex] 
\end{tabular} 
\end{table} 
[1] "Text"

            coeff_c coeff_f stderr_c stderr_f pvalues_c pvalues_f zvalues_c zvalues_f
-------------------------------------------------------------------------------------
(Intercept)  0.595   0.595   0.021    0.021       0         0      28.465    2

### Logistic

#### Centralized

In [6]:
# Housekeeping
if(exists('results_centralized')){
    rm(results_centralized)
}
if(exists('results_federated_plain')){
    rm(results_federated_plain)
}

datasets <- list(
  read.csv('../data/logistic_party1.csv'),
  read.csv('../data/logistic_party2.csv'),
  read.csv('../data/logistic_party3.csv')
)
datasets_combined <- do.call(rbind.data.frame, datasets)
results_centralized <- glm(data=datasets_combined, formula = y ~ x1 + x2, family=binomial(link = "logit"))

#### Federated

In [7]:
client <- vtg::MockClient$new(datasets, "vtg.glm")
results_federated_plain <- vtg.glm::dglm(client, formula = y ~ x1 + x2, family=binomial(link = "logit"), tol=1e-08, maxit=25)

DEBUG [11:56:31.097] Initializing... 
INFO  [11:56:31.100]  
INFO  [11:56:31.102] ############################################### 
INFO  [11:56:31.104] # Starting iteration 0 
INFO  [11:56:31.107] ############################################### 
INFO  [11:56:31.110]  
INFO  [11:56:31.112] 0.1 - RPC Node Beta 
DEBUG [11:56:31.115] ** Mocking call to "node_beta" ** 
DEBUG [11:56:31.117] Regular call 
DEBUG [11:56:31.120] there are 3 datasets ..  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:31.138] Log for site 1: 
DEBUG [11:56:31.141] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [11:56:31.126] Initializing node beta... 
DEBUG [11:56:31.133] First iteration. Initializing variables. 
DEBUG [11:56:31.135] Calculating the Betas.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:31.156] Log for site 2: 
DEBUG [11:56:31.158] Running a (mere) regular container. 
Calling RPC_node_beta 
DEBUG [11:56:31.144] Initializing node beta... 
DEBUG [11:56:31.150] First iteration. Initializing variables. 
DEBUG [11:56:31.153] Calculating the Betas.  


...

DEBUG [11:56:32.161] Log for site 1: 
DEBUG [11:56:32.164] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:32.149] Starting node deviance. 
DEBUG [11:56:32.158] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:32.182] Log for site 2: 
DEBUG [11:56:32.184] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:32.167] Starting node deviance. 
DEBUG [11:56:32.178] Updating the variables for node deviance.  


Warning message in if (family == "rs.poi") eval(as.name(dstar), data) else dstar:
"the condition has length > 1 and only the first element will be used"


DEBUG [11:56:32.198] Log for site 3: 
DEBUG [11:56:32.201] Running a (mere) regular container. 
Calling RPC_node_deviance 
DEBUG [11:56:32.187] Starting node deviance. 
DEBUG [11:56:32.194] Updating the variables for node deviance.  
DEBUG [11:56:32.203]   - [DONE] 
INFO  [11:56:32.206] 5.4 - Master deviance 
DEBUG [11:56:32.210] Starting master deviance. 
DEBUG [11:56:32.213]   - [DONE] 
INFO  [11:56:32.215] 5.5 - Termination conditions 
DEBUG [11:56:32.218]   - [CONVERGED] 
DEBUG [11:56:32.221] Preparing output 


#### Comparison

In [8]:
val_binomial_res = validate(results_centralized, results_federated_plain)
print(val_binomial_res)

[1] "LaTeX"

% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Thu, May 05, 2022 - 11:56:32
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}} ccccccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & coeff\_c & coeff\_f & stderr\_c & stderr\_f & pvalues\_c & pvalues\_f & zvalues\_c & zvalues\_f \\ 
\hline \\[-1.8ex] 
(Intercept) & $0.104$ & $0.104$ & $0.064$ & $0.064$ & $0.106$ & $0.106$ & $1.617$ & $1.617$ \\ 
x1 & $2.593$ & $2.593$ & $0.093$ & $0.093$ & $0$ & $0$ & $27.802$ & $27.802$ \\ 
x2 & $$-$0.054$ & $$-$0.054$ & $0.051$ & $0.051$ & $0.291$ & $0.291$ & $$-$1.055$ & $$-$1.055$ \\ 
\hline \\[-1.8ex] 
\end{tabular} 
\end{table} 
[1] "Text"

            coeff_c coeff_f stderr_c stderr_f pvalues_c pvalues_f zvalues_c zvalues_f
-------------------------------------------------------------------------------------
(Intercept)  0.104   0.104   0.064    0.064     0.106 

### Custom relative survival (with Poisson error)

In [12]:
# Housekeeping
if(exists('results_centralized')){
    rm(results_centralized)
}
if(exists('results_federated_plain')){
    rm(results_federated_plain)
}

modperiod.link <- function(dstar) {
  structure(
    list(
      # Link
      linkfun = function(mu) {log(mu - dstar)} ,
        
      # Inverse link
      linkinv = function(eta) {exp(eta) + dstar} ,
        
      # Derivative of the inverse link (d_mu/d_eta)
      mu.eta = function(eta) {exp(eta)},
        
      # Functions for domain checking
      valideta = function(eta) TRUE,
      validmu = function(mu) mu > dstar ,
        
      #validmu = function(mu) all(is.finite(mu)) && all(mu > 0),
      name = "modperiod.link"
    ),
    class = "link-glm"
  )
}
format_data <- function(data, types) {
  
  column_names = names(types)
  for(i in 1:length(types)) {
    column_name = column_names[i]
    specs = types[[i]]
    type_ = specs$type
    if (type_ == "numeric"){
      data[[column_name]] = as.numeric(data[[column_name]])
    }
    if (type_ == "factor"){
      data = data[data[[column_name]] %in% specs$levels,]
      data[[column_name]] = factor(data[[column_name]], levels=specs$levels)
      if(! is.null(specs$ref)) data[[column_name]] = relevel(data[[column_name]], ref=specs$ref)
    }
  }
  data
}
types=list(prog=list(type='factor', levels=c('Vocational', 'General', 'Academic'), ref=NULL))
data <- format_data(datasets_combined, types)
mustart = pmax(data$y, data$y) + 0.1

results_centralized <- glm(data=data, formula=y ~ x1 + x2 + x3, family = poisson(link = modperiod.link(data$y)), mustart=mustart)

client <- vtg::MockClient$new(datasets, "vtg.glm")
results_federated_plain <- vtg.glm::dglm(client, formula=y ~ x1 + x2 + x3, types=types, family='rs.poi', dstar="x2", tol=1e-08, maxit=25)

val_customglm_res = validate(results_centralized, results_federated_plain)
print(val_customglm_res)
rm(results_centralized)
rm(results_federated_plain)

ERROR: Error in eval(predvars, data, env): object 'x3' not found


## References
For more information, please refer to our paper:

> M. Cellamare, A.J. van Gestel, H. Alradhi, F. Martin, A. Moncada-Torres. "A Federated Generalized Linear Model for Privacy-Preserving Analysis". Under review